copy from: E:\Github\TensorFlow-Examples\examples\2_BasicModels\kmeans.py

@资料：

tensorflow 5. K均值回归用于手写数字识别 https://blog.csdn.net/yuanlulu/article/details/79742318


@识别精度：

k = 25 # The number of clusters #71.27%

k = 250 #91.89%

k = 2500 #95.61%


In [105]:
""" K-Means.

Implement K-Means algorithm with TensorFlow, and apply it to classify
handwritten digit images. This example is using the MNIST database of
handwritten digits as training samples (http://yann.lecun.com/exdb/mnist/).

Note: This example requires TensorFlow v1.1.0 or over.

Author: Aymeric Damien
Project: https://github.com/aymericdamien/TensorFlow-Examples/
"""

from __future__ import print_function

import numpy as np
import tensorflow as tf
from tensorflow.contrib.factorization import KMeans

# Ignore all GPUs, tf k-means does not benefit from it.
import os
os.environ["CUDA_VISIBLE_DEVICES"] = ""

# Import MNIST data
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("/tmp/data/", one_hot=True)
full_data_x = mnist.train.images

# Parameters
num_steps = 10 # Total steps to train
batch_size = 1024 # The number of samples per batch #未用过
k = 25 # The number of clusters #71.27%
# k = 250 #91.89%
# k = 2500 #95.61%
num_classes = 10 # The 10 digits
num_features = 784 # Each image is 28x28 pixels

# Input images
X = tf.placeholder(tf.float32, shape=[None, num_features], name='ph_x')
# Labels (for assigning a label to a centroid and testing)
Y = tf.placeholder(tf.float32, shape=[None, num_classes], name='ph_y')

# K-Means Parameters
kmeans = KMeans(inputs=X, num_clusters=k, distance_metric='cosine',
                use_mini_batch=True)

# Build KMeans graph
training_graph = kmeans.training_graph()
print('len(training_graph):', len(training_graph))

if len(training_graph) > 6: # Tensorflow 1.4+
    (all_scores, cluster_idx, scores, cluster_centers_initialized,
     cluster_centers_var, init_op, train_op) = training_graph
else:
    (all_scores, cluster_idx, scores, cluster_centers_initialized,
     init_op, train_op) = training_graph

cluster_idx = cluster_idx[0] # fix for cluster_idx being a tuple
avg_distance = tf.reduce_mean(scores)

# Initialize the variables (i.e. assign their default value)
init_vars = tf.global_variables_initializer()

# Start TensorFlow session
sess = tf.Session()

# Run the initializer
sess.run(init_vars, feed_dict={X: full_data_x})
sess.run(init_op, feed_dict={X: full_data_x})

# Training
for i in range(1, num_steps + 1):
    _, d, idx = sess.run([train_op, avg_distance, cluster_idx],
                         feed_dict={X: full_data_x})
    if i % 10 == 0 or i == 1:
        print("Step %i, Avg Distance: %f" % (i, d))

# Assign a label to each centroid
# Count total number of labels per centroid, using the label of each training
# sample to their closest centroid (given by 'idx')
counts = np.zeros(shape=(k, num_classes))
for i in range(len(idx)):
    counts[idx[i]] += mnist.train.labels[i]
# Assign the most frequent label to the centroid
labels_map = [np.argmax(c) for c in counts] #int32
print('labels_map:', labels_map)
# labels_map = np.array([np.argmax(c) for c in counts]) #注意会变成 int64
# print('labels_map.shape:', np.array(labels_map).shape) #25*1
labels_map = tf.convert_to_tensor(labels_map)
print('labels_map.shape:', labels_map.shape, labels_map)

# Evaluation ops
# Lookup: centroid_id -> label
cluster_label = tf.nn.embedding_lookup(labels_map, cluster_idx)
# Compute accuracy
correct_prediction = tf.equal(cluster_label, tf.cast(tf.argmax(Y, 1), tf.int32))
# correct_prediction = tf.equal(cluster_label, tf.argmax(Y, 1) ) #报错：TypeError: Input 'y' of 'Equal' Op has type int64 that does not match type int32 of argument 'x'.

accuracy_op = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

# Test Model
test_x, test_y = mnist.test.images, mnist.test.labels #10000x784; 10000x10
print("Test Accuracy:", sess.run(accuracy_op, feed_dict={X: test_x, Y: test_y}))


Extracting /tmp/data/train-images-idx3-ubyte.gz
Extracting /tmp/data/train-labels-idx1-ubyte.gz
Extracting /tmp/data/t10k-images-idx3-ubyte.gz
Extracting /tmp/data/t10k-labels-idx1-ubyte.gz
len(training_graph): 6
Step 1, Avg Distance: 0.341471
Step 10, Avg Distance: 0.221609
labels_map: [3, 1, 8, 6, 2, 6, 4, 5, 7, 9, 3, 5, 2, 4, 8, 0, 0, 6, 7, 1, 2, 3, 6, 1, 5]
labels_map.shape: (25,) Tensor("Const_61:0", shape=(25,), dtype=int32)
Test Accuracy: 0.6953


In [106]:
'''
报错：InvalidArgumentError: You must feed a value for placeholder tensor 'ph_x' with dtype float and shape [?,784]
说明 cluster_idx 是一个含placeholder X 的node

cluster_idx:   <tf.Tensor 'Squeeze_39:0' shape=<unknown> dtype=int64>
all_scores[0]: <tf.Tensor 'sub_81:0' shape=(?, ?) dtype=float32>
scores[0]:     <tf.Tensor 'Squeeze_40:0' shape=<unknown> dtype=float32>
'''
sess.run(cluster_idx)
# sess.run(all_scores[0]) #同理



InvalidArgumentError: You must feed a value for placeholder tensor 'ph_x' with dtype float and shape [?,784]
	 [[Node: ph_x = Placeholder[_class=["loc:@Squeeze_41"], dtype=DT_FLOAT, shape=[?,784], _device="/job:localhost/replica:0/task:0/device:CPU:0"]()]]

Caused by op 'ph_x', defined at:
  File "D:\Anaconda3\envs\python356-ana3\lib\runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "D:\Anaconda3\envs\python356-ana3\lib\runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "D:\Anaconda3\envs\python356-ana3\lib\site-packages\ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "D:\Anaconda3\envs\python356-ana3\lib\site-packages\traitlets\config\application.py", line 658, in launch_instance
    app.start()
  File "D:\Anaconda3\envs\python356-ana3\lib\site-packages\ipykernel\kernelapp.py", line 499, in start
    self.io_loop.start()
  File "D:\Anaconda3\envs\python356-ana3\lib\site-packages\tornado\platform\asyncio.py",

In [103]:
training_graph

((<tf.Tensor 'sub_77:0' shape=(?, ?) dtype=float32>,),
 (<tf.Tensor 'Squeeze_39:0' shape=<unknown> dtype=int64>,),
 (<tf.Tensor 'Squeeze_38:0' shape=<unknown> dtype=float32>,),
 <tf.Variable 'initialized_19:0' shape=() dtype=bool_ref>,
 <tf.Tensor 'cond_19/Merge:0' shape=() dtype=bool>,
 <tf.Operation 'group_deps_19' type=NoOp>)

In [113]:
test_x, test_x.shape, test_y, test_y.shape

(array([[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]], dtype=float32),
 (10000, 784),
 array([[0., 0., 0., ..., 1., 0., 0.],
        [0., 0., 1., ..., 0., 0., 0.],
        [0., 1., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]]),
 (10000, 10))

In [114]:
cluster_label

<tf.Tensor 'embedding_lookup_26:0' shape=<unknown> dtype=int32>

In [57]:
'''
https://www.jianshu.com/p/677e71364c8e
tf.nn.embedding_lookup 原理

https://blog.csdn.net/UESTC_C2_403/article/details/72779417
tf.nn.embedding_lookup函数的用法
'''
embedding = tf.Variable(np.identity(6, dtype=np.int32))
input_ids = tf.placeholder(dtype=tf.int32, shape=[None])
input_embedding = tf.nn.embedding_lookup(embedding, input_ids)

sess.run(tf.global_variables_initializer())
# sess.run(tf.initialize_all_variables()) #弃用：WARNING: initialize_all_variables (from tensorflow.python.ops.variables) is deprecated and will be removed after 2017-03-02.

print(sess.run(embedding))
print(sess.run(input_embedding, feed_dict={input_ids: [4, 0, 2, 4, 5, 1, 3, 0]}))


c = np.random.random([10,1])
b = tf.nn.embedding_lookup(c, [1, 3])
print(sess.run(b))
print(c)

[[1 0 0 0 0 0]
 [0 1 0 0 0 0]
 [0 0 1 0 0 0]
 [0 0 0 1 0 0]
 [0 0 0 0 1 0]
 [0 0 0 0 0 1]]
[[0 0 0 0 1 0]
 [1 0 0 0 0 0]
 [0 0 1 0 0 0]
 [0 0 0 0 1 0]
 [0 0 0 0 0 1]
 [0 1 0 0 0 0]
 [0 0 0 1 0 0]
 [1 0 0 0 0 0]]
[[0.49019589]
 [0.72128981]]
[[0.88267262]
 [0.49019589]
 [0.90205023]
 [0.72128981]
 [0.29841324]
 [0.05815602]
 [0.96688145]
 [0.35316167]
 [0.00357044]
 [0.90402594]]


In [48]:
tf.nn.embedding_lookup?

In [44]:
idx, idx.shape

(array([13, 10,  9, ..., 24,  3,  2], dtype=int64), (55000,))

In [30]:
d

0.2191542

In [42]:
print(repr(counts[0]))
print(np.argmax(counts[0]))

array([  25.,    0.,   33., 1272.,    0.,  371.,    3.,    5.,   55.,
          3.])
3


In [104]:
KMeans.training_graph?

In [7]:
tf.reduce_mean?

In [126]:
'sess' in locals()

True

In [127]:
sess.close()

In [131]:
!nvidia-smi

Fri Mar 15 09:56:20 2019       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 385.54                 Driver Version: 385.54                    |
|-------------------------------+----------------------+----------------------+
| GPU  Name            TCC/WDDM | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  GeForce GTX 960    WDDM  | 00000000:01:00.0  On |                  N/A |
|  0%   61C    P2    27W / 160W |   1807MiB /  2048MiB |      1%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:                                                       GPU Memory |
|  GPU  